In [145]:
import pandas as pd
import numpy as np
import operator

In [29]:
def create_entity_index(blocks, entities):
    entity_index = {}
    for entity in entities:
        entity_index[entity] = []
    for key, value in blocks.items():
        for entity in value:
            entity_index[entity].append(key)
    return entity_index

In [71]:
def Scheme(blocks_i, entity_i, blocks_j, entity_j, type_of_scheme="Common"):
    inter_blocks_ij = []
    for block in blocks_i:
        if entity_j in block and entity_i in block:
            inter_blocks_ij.append(block)
    if type_of_scheme == "Jaccard":
        return len(inter_blocks_ij) / (len(blocks_i) + len(blocks_j) - len(inter_blocks_ij))
    return len(inter_blocks_ij)

In [103]:
blocks = {'tt0427309': ['164_kb1', '164_kb2'],
 'Weinstein': ['164_kb1', '164_kb2', '166_kb1'],
 'Marshall': ['164_kb1', '164_kb2', '165_kb1'],
    'Kin': ['165_kb1', '164_kb2']}
entity_index = create_entity_index(a, ['165_kb1','164_kb1', '164_kb2', '166_kb1'])

In [117]:
def make_graph(blocks, type_of_scheme="Jaccard"):
    E = []
    V = set()
    W = []
    for block in blocks:
        ps_1 = [p for p in blocks[block] if p[-3:] == "kb1"]
        ps_2 = [p for p in blocks[block] if p[-3:] == "kb2"]
        for p1 in ps_1:
            V.add(p1)
            for p2 in ps_2:
                V.add(p2)
                blocks_1_id = entity_index[p1]
                blocks_2_id = entity_index[p2]
                blocks_1 = [blocks[id1] for id1 in blocks_1_id]
                blocks_2 = [blocks[id2] for id2 in blocks_2_id]
                weight = Scheme(blocks_1, p1, blocks_2, p2, type_of_scheme)
                E.append((p1,p2))
                W.append(weight)            
    norm_W = [(float(i)-min(W))/(max(W)-min(W)) for i in W]
    return V, E, norm_W

In [142]:
graph = make_graph(a)

In [143]:
def WEP(graph):
    V = graph[0]
    E = graph[1]
    W = graph[2]
    W_min = np.mean(W)
    low_W_index = []
    for i in range(0,len(E)):
        if W[i] < W_min:
            low_W_index.append(i)
    E = np.delete(E, low_W_index, axis=0)       
    W = np.delete(W, low_W_index)
    return V, E, W

In [144]:
pruned_graph = WEP(graph)

In [169]:
def CEP(graph, K=4):
    V = graph[0]
    E = graph[1]
    W = graph[2]
    sorted_stack = {}
    for i in range(0,len(E)):
        sorted_stack[i] = W[i]
        sorted_stack = {k: v for k, v in sorted(sorted_stack.items(), key=lambda item: item[1], reverse=True)}
        
        if K < len(sorted_stack):
            sorted_stack.popitem()
    retain_index = sorted_stack.keys()

    E = [E[i] for i in retain_index] 
    W = [W[i] for i in retain_index] 
    return V, E, W

In [172]:
pruned_graph_2 = CEP(graph)

In [173]:
print(pruned_graph_2[0])
print(pruned_graph_2[1])
print(pruned_graph_2[2])

{'164_kb1', '166_kb1', '165_kb1', '164_kb2'}
[('164_kb1', '164_kb2'), ('164_kb1', '164_kb2'), ('164_kb1', '164_kb2'), ('165_kb1', '164_kb2')]
[1.0, 1.0, 1.0, 0.5]
